In [110]:
import pandas as pd
import datetime as dt 
import numpy as np
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
import time
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif
import plotly.graph_objs as go
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import ensemble
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import shap
import pandas_ta as ta


import pandas_datareader as pdr
import yfinance as yf
import datetime as dt


In [112]:
aapl_df = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "AAPL",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "6mo",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1h",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

[*********************100%***********************]  1 of 1 completed


In [113]:
aapl_df

,Open,High,Low,Close,Volume
2021-10-25 04:00:00-04:00,148.500000,148.900000,148.41,148.720000,0
2021-10-25 05:00:00-04:00,148.720000,148.730000,148.47,148.580000,0
2021-10-25 06:00:00-04:00,148.550000,148.630000,148.48,148.500000,0
2021-10-25 07:00:00-04:00,148.500000,148.550000,148.11,148.440000,0
2021-10-25 08:00:00-04:00,148.440000,148.640000,148.13,148.460000,0
...,...,...,...,...,...
2022-04-22 15:30:00-04:00,163.130005,164.160004,161.50,161.830002,12876649
2022-04-22 16:00:00-04:00,161.830000,168.880000,161.20,161.500000,0
2022-04-22 17:00:00-04:00,161.500000,166.310000,161.30,161.340000,0
2022-04-22 18:00:00-04:00,161.340000,166.170000,161.11,161.320000,0


In [114]:
aapl_df['day_change'] = aapl_df['Close'].pct_change()

In [115]:
aapl_df = aapl_df[1:]

k = aapl_df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()
# Get the 12-day EMA of the closing price
d = aapl_df['Close'].ewm(span=26, adjust=False, min_periods=26).mean()
# Subtract the 26-day EMA from the 12-Day EMA to get the MACD
macd = k - d

aapl_df['Macd'] = macd
aapl_df






A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Open,High,Low,Close,Volume,day_change,Macd
2021-10-25 05:00:00-04:00,148.720000,148.730000,148.47,148.580000,0,-0.000941,NaN
2021-10-25 06:00:00-04:00,148.550000,148.630000,148.48,148.500000,0,-0.000538,NaN
2021-10-25 07:00:00-04:00,148.500000,148.550000,148.11,148.440000,0,-0.000404,NaN
2021-10-25 08:00:00-04:00,148.440000,148.640000,148.13,148.460000,0,0.000135,NaN
2021-10-25 09:00:00-04:00,148.450000,148.750000,148.38,148.690000,0,0.001549,NaN
...,...,...,...,...,...,...,...
2022-04-22 15:30:00-04:00,163.130005,164.160004,161.50,161.830002,12876649,-0.008091,-1.173911
2022-04-22 16:00:00-04:00,161.830000,168.880000,161.20,161.500000,0,-0.002039,-1.335965
2022-04-22 17:00:00-04:00,161.500000,166.310000,161.30,161.340000,0,-0.000991,-1.460470
2022-04-22 18:00:00-04:00,161.340000,166.170000,161.11,161.320000,0,-0.000124,-1.542968


In [116]:
aapl_df = aapl_df.dropna()

In [118]:
data = aapl_df.to_numpy() 

train_data = [[row[3], row[6]] for row in data]

change = [row[5] for row in data][:500]

x = data[:500]

xTe = data[500:]


def dataset(pct_change):
  y = []
  for i in pct_change:
    if i > 0.01:
      y.append(1)
    elif i < 0.01:
      y.append(-1)
    else:
      y.append(0)
  return y


y = dataset(change)





X_train, X_valid, y_train, y_valid = train_test_split(x,y,test_size=.2, shuffle=False) # Using 80% for training, 20% validation





# model1 = KMeans(n_clusters=2)

model1 = GaussianNB(var_smoothing= 0.8111308307896871)


# GRID SEARCH

# model 1
params = [{"var_smoothing": np.logspace(0,-9,num=100)}]
gsearch1 = GridSearchCV(estimator=model1, cv=TimeSeriesSplit(max_train_size=500)
, param_grid=params)
gsearch1.fit(X_train, y_train)
print(gsearch1.best_params_)



model1.fit(X_train, y_train)


y_val_pred = model1.predict(X_valid)



rand_score = metrics.rand_score(y_valid, y_val_pred)



# print(rand_score)

# metrics.silhouette_score(X_train, y_train, metric='euclidean', sample_size=None, random_state=None)
recall_valid = metrics.recall_score(y_valid,y_val_pred,average=None, zero_division=True)
precision_valid = metrics.precision_score(y_valid,y_val_pred,average=None, zero_division=True)
accuracy_valid = metrics.accuracy_score(y_valid,y_val_pred)
print("Model1: ")
print(recall_valid)
print(precision_valid)
print(accuracy_valid, "\n")









{'var_smoothing': 0.8111308307896871}
Model1: 
[0.94736842 0.2       ]
[0.95744681 0.16666667]
0.91 

